## Opik configuration Guide

This notebook demonstrates the basic usage of the `opik` library. We'll cover:

- Logging test cases  
- Running evaluations  
- Viewing and saving results locally  
- Evaluating Opik metrics through the Trace metrics API


In [ ]:
!pip install opik

In [ ]:
import os
OPENAI_API_KEY = "your_openai_api_key_here"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

### Metric Evaluation Using `opik.evaluation.metrics`

This script evaluates the similarity and moderation quality of a generated text (`output`) compared to a reference text (`reference`) using the `opik.evaluation.metrics` package.

#### Inputs

- **Output**: A sample response that might come from a language model.
- **Reference**: A human-written or expected answer used for comparison.

#### Metrics Used

- **Equals**: Checks if the output and reference are equal, ignoring case sensitivity.
- **Moderation**: Evaluates the moderation compliance of the output (e.g., safety, appropriateness).

#### Processing Flow

1. A list of metric instances is created.
2. Each metric is applied to the `output` and `reference`.
3. The result (`.value`) from each metric is stored in a dictionary for easy access.


In [4]:
from opik.evaluation.metrics import Equals, Moderation, GEval
from datetime import datetime, timezone

# Example output and reference
output = """Paris is the capital of France and one of the most visited cities in the world. 
While some tourists express concerns about safety in certain neighborhoods, Paris remains a vibrant and welcoming city. 
Visitors are advised to stay vigilant, especially in crowded areas, but overall, the city is considered safe for travelers."""
reference = """Paris is the capital of France and a major tourist destination. 
While no city is entirely without risk, Paris is generally safe for visitors who take standard precautions."""

metrics = [
    Equals(case_sensitive=False),
    Moderation()
]

metric_results = {}
for m in metrics:
    if isinstance(m, Equals):
        result = m.score(output=output, reference=reference)
    elif isinstance(m, Moderation):
        result = m.score(output=output, reference=reference)
    else:
        continue
    metric_results[m.__class__.__name__] = result.value



In [5]:
print("Metric Results:", metric_results)

Metric Results: {'Equals': 0.0, 'Moderation': 0.0}


## How to get your subscription key for the TRACE Metrics API

To access the TRACE Metrics API, you need a **subscription key**. Follow these steps:

1. **Sign in to CognitiveView**  
   Go to [app.cognitiveview.com](https://app.cognitiveview.com) and log in with your account credentials.

2. **Navigate to System Settings**  
   Once logged in, find the **System Settings** option in the main menu.

3. **Generate and view your subscription key**  
   - Inside **System Settings**, look for the section labeled **API Access** or **Subscription Key**.
   - If a key has already been generated, you can copy it directly.
   - If not, click the **Generate Key** button to create a new key.

4. **Copy and save your subscription key**  
   Keep this key secure. You’ll need to include it in your API requests for authentication.

---

## Posting Evaluation Metrics to TRACE Metrics API

This script sends your evaluation metric results (for example, from DeepEval) to the TRACE Metrics API using an authenticated HTTP POST request.

### Authentication

- Use an **Authorization token** (`AUTH_TOKEN`) in the request header.


---

### Endpoint

| Item         | Value                                                      |
|--------------|-----------------------------------------------------------|
| **Base URL** | `https://api.cognitiveview.com`                            |
| **API Path** | `/metrics`                                          |
| **Full URL** | `https://api.cognitiveview.com/metrics`             |

---

### Payload Structure

#### `metric_metadata`

Contains information about what you’re evaluating:
- `application_name`: Name of the evaluated application.
- `version`: Version of the application or model.
- `provider`: The metric system or platform (for example, `deepeval`).
- `use_case`: The business or functional domain (for example, `customer_support`).


In [ ]:
import requests

BASE_URL = "https://api.cognitiveview.com"
AUTH_TOKEN ="Your Token"  # Replace with your actual token
url = f"{BASE_URL}/metrics"

headers = {
    "Ocp-Apim-Subscription-Key": AUTH_TOKEN,
    "Content-Type": "application/json",
}

payload = {
  "metric_metadata": {
    "application_name": "chat-application",
    "version": "1.0.0",
    "eval_provider": "opik",
    "use_case": "transportation"
  },
  "metric_data": {
    "opik": metric_results
  } 
}

response = requests.post(url, headers=headers, json=payload)

# Output the response
print(f"Status Code: {response.status_code}")
print("Response JSON:", response.json())